In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
from tqdm import tqdm_notebook

In [0]:
SessionLink = 'https://ipp-ean20.netkey.at/index.php?p=elasearch&t=browsesessions&pnr='
TotalPages = 10

In [0]:
i = 1
url = SessionLink + str(i)

In [0]:
response = requests.get(url)
data = BeautifulSoup(response.text,'lxml')

In [0]:
results = data.find('div',class_ = 'ipp-result-list gallery')
SessionList = results.find_all('div',class_ = 'ipp-result-list-item-info-title')

In [0]:
SessionLinks = []
for session in SessionList:
    SessionLinks.append(session.find('a')['href'])

In [0]:
len(SessionLinks)

28

In [5]:
SessionLinks = []
numberSessions = 0
for i in range(1,TotalPages+1):
    url = SessionLink + str(i)
    response = requests.get(url)
    data = BeautifulSoup(response.text,'lxml')
    results = data.find('div',class_ = 'ipp-result-list gallery')
    SessionList = results.find_all('div',class_ = 'ipp-result-list-item-info-title')
    for session in SessionList:
        SessionLinks.append(session.find('a')['href'])
    # numberSessions = len(SessionLinks) - numberSessions
    print(len(SessionList))
    

28
32
27
25
27
26
25
27
31
0


In [6]:
len(SessionLinks)

248

In [0]:
sessUrl = 'https://ipp-ean20.netkey.at/index.php?p=recorddetail&rid=0ae88416-b0d9-4b76-8747-23b364b59038&t=browsesessions'
sessionResp = requests.get(sessUrl)
sessionInfo = BeautifulSoup(sessionResp.text,'lxml')

In [11]:
SessionDetails = sessionInfo.find('div',class_ = 'ipp-record-detail-infos')
SessionDetails

<div class="ipp-record-detail-infos">
<div class="ipp-record-detail-title">
                TC01 - Antibodies: from autoimmune encephalitis to demyelinating myelopathies -  Level 2            </div>
<div class="ipp-record-detail-info-row">
<span class="label label-default">Neuroimmunology</span>
</div>
<div class="ipp-record-detail-info-row">
<span class="ipp-record-detail-datetime">
                        Saturday, May 23,                                                    15:00
                                                            - 18:15
                                                                        </span>
</div>
<div class="ipp-record-detail-info-row">
<span class="ipp-record-detail-room">
<strong>Room:</strong> Main Auditorium

                    </span>
</div>
<div class="ipp-record-detail-info-row">
<span class="ipp-record-detail-room">
<strong>Type of session:</strong> Teaching Course                    </span>
</div>
<div class="ipp-record-detail-info-row">
<

In [12]:
SessionDict = {}
SessionDict['Session Heading'] = SessionDetails.find('div',class_ = 'ipp-record-detail-title').text.strip()
SessionDict['Session Time'] = SessionDetails.find('span',class_ = 'ipp-record-detail-datetime').text.replace(' ','').strip()
try:
    SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[0].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[0].text.strip()
except:
    pass
try:
    SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[1].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[1].text.strip()
except:
    pass
try:
    SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[2].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[2].text.strip()
except:
    pass
try:
    SessionDict['Session Description'] = sessionInfo.find('div',class_ = 'ipp-record-detail-description').text
except:
    pass
print(SessionDict)

{'Session Heading': 'TC01 - Antibodies: from autoimmune encephalitis to demyelinating myelopathies -  Level 2', 'Session Time': 'Saturday,May23,15:00\n-18:15', 'Room:': 'Room: Main Auditorium', 'Type of session:': 'Type of session: Teaching Course', 'Topic:': 'Topic: Neuroimmunology', 'Session Description': '\nDescription\nIn this TC, the speakers will provide practical relevant knowledge on autoantibody testing in neurological disorders, as well as basics to understand immunology and classify the different diseases. They will discuss the clinical phenotypes, suggested treatments and discuss the newest developments in this rapidly advancing field of neurology. They will provide tricks to recognize these diseases and share pitfalls to avoid misdiagnosis. In addition, the attendant will learn how these diseases can be relevant to other disciplines within clinical neurology and basic neurosciences\n'}


In [14]:
PresentationDetails = sessionInfo.find('div',class_ = 'ipp-record-detail-children')
try:
    Presentations = PresentationDetails.find_all('div',class_ = 'ipp-record-detail ipp-record-detail-child ')
    print(Presentations)
except:
    Presentations = []

[<div class="ipp-record-detail ipp-record-detail-child " id="ipp-record-detail-939cfdac-87f4-4512-97d9-f324b4c155d9">
<div class="ipp-record-detail-child-number">
<div class="ipp-record-detail-number">TC01_01</div> </div>
<div class="ipp-record-detail-child-body">
<div class="ipp-record-detail-title">
            Basics of neuroimmunology        </div>
<div class="ipp-record-detail-author-block">
<u>S. Bittner</u>; Mainz/DE
            </div>
<div class="ipp-record-detail-actions">
<a class="btn btn-default ipp-record-detail-action-add-to-schedule" href="javascript:addToBasket('939cfdac-87f4-4512-97d9-f324b4c155d9');"> <i aria-hidden="true" class="fa fa-plus"></i> My schedule</a>
<a class="btn btn-primary ipp-record-detail-action-remove-from-schedule" href="javascript:removeFromBasket('939cfdac-87f4-4512-97d9-f324b4c155d9');"> <i aria-hidden="true" class="fa fa-minus"></i> My schedule</a>
</div>
</div>
</div>, <div class="ipp-record-detail ipp-record-detail-child " id="ipp-record-detai

In [0]:
try:
    PresentationCode = Presentations[0].find('div',class_ = 'ipp-record-detail-child-number').text.strip()
except:
    PresentationCode = ''
try:
    PresentationTitle = Presentations[0].find('div',class_ = 'ipp-record-detail-title').text.strip()
except:
    PresentationTitle = ''
try:    
    PresentationAuthors = Presentations[0].find('div',class_ = 'ipp-record-detail-author-block').text.strip()
except:
    PresentationAuthors = ''
try:
    PresentationAbstract = Presentations[0].find('div',class_ = 'ipp-record-detail-abstract-content').text.strip()
except:
    PresentationAbstract = ''
print(PresentationCode)
print(PresentationTitle)
print(PresentationAuthors)
print(PresentationAbstract)

In [21]:
Masterdf = pd.DataFrame()

for i in tqdm_notebook(SessionLinks):
    sessUrl = i
    sessionResp = requests.get(sessUrl)
    sessionInfo = BeautifulSoup(sessionResp.text,'lxml')
    SessionDetails = sessionInfo.find('div',class_ = 'ipp-record-detail-infos')
    SessionDict = {}
    SessionDict['Session Heading'] = SessionDetails.find('div',class_ = 'ipp-record-detail-title').text.strip()
    SessionDict['Session Time'] = SessionDetails.find('span',class_ = 'ipp-record-detail-datetime').text.replace(' ','').strip()
    try:
        SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[0].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[0].text.strip()
    except:
        pass
    try:
        SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[1].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-room')[1].text.strip()
    except:
        pass
    try:
        SessionDict[SessionDetails.find_all('span',class_ = 'ipp-record-detail-moderators')[0].strong.text] = SessionDetails.find_all('span',class_ = 'ipp-record-detail-moderators')[0].text.strip()
    except:
        pass
    try:
        SessionDict['Session Description'] = sessionInfo.find('div',class_ = 'ipp-record-detail-description').text
    except:
        pass
    PresentationDetails = sessionInfo.find('div',class_ = 'ipp-record-detail-children')
    try:
        Presentations = PresentationDetails.find_all('div',class_ = 'ipp-record-detail ipp-record-detail-child ')
        print("Presentations",len(Presentations))
#         print(Presentations)
    except:
        print("Exception")
        print(i)
        Presentations = []
#     presentationDict = SessionDict.copy()
    if(len(Presentations)==0):
        # print(i)
        Masterdf = Masterdf.append(SessionDict,ignore_index = True)
    else:
        for i in tqdm_notebook(range(len(Presentations))):
            presentationDict = SessionDict.copy()
            try:
                PresentationCode = Presentations[i].find('div',class_ = 'ipp-record-detail-child-number').text.strip()
            except:
                PresentationCode = ''
            try:
                PresentationTitle = Presentations[i].find('div',class_ = 'ipp-record-detail-title').text.strip()
            except:
                PresentationTitle = ''
            try:    
                PresentationAuthors = Presentations[i].find('div',class_ = 'ipp-record-detail-author-block').text.strip()
            except:
                PresentationAuthors = ''
            try:
                PresentationAbstract = Presentations[i].find('div',class_ = 'ipp-record-detail-abstract-content').text.strip()
            except:
                PresentationAbstract = ''
            presentationDict['Presentation Code'] = PresentationCode
            presentationDict['Presentation Title'] = PresentationTitle
            presentationDict['Presentation Authors'] = PresentationAuthors
            presentationDict['Presentation Abstract'] = PresentationAbstract
            Masterdf = Masterdf.append(presentationDict,ignore_index = True)
Masterdf    



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


Exception
https://ipp-ean20.netkey.at/index.php?p=recorddetail&rid=9e0435a0-10e1-4165-8a3c-8e3ce0abbbf6&t=browsesessions
Presentations 3


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Presentations 3



Presentations 3


...


Presentations 4



Presentations 4



Presentations 4



Presentations 4


,Room:,Session Description,Session Heading,Session Time,Topic:,Type of session:,Presentation Abstract,Presentation Authors,Presentation Code,Presentation Title
0,Room: Vienna,\nDescription\nMechanical thrombectomy revolut...,FW02 - Overarching theme - Can we protect and ...,"Saturday,May23,08:00\n-09:30","Topic: Cerebrovascular diseases, Overarching t...",Type of session: Focused Workshop,NaN,NaN,NaN,NaN
1,Room: CbW Room,\nDescription\nThis case-based workshop is int...,CbW01 - Continuous EEG in critically ill patients,"Saturday,May23,08:00\n-09:30",Topic: Epilepsy,Type of session: Case-based Workshop,,S. Beniczky; Dianalund/DK,CBW01_01,Nonconvulsive status epilepticus
2,Room: CbW Room,\nDescription\nThis case-based workshop is int...,CbW01 - Continuous EEG in critically ill patients,"Saturday,May23,08:00\n-09:30",Topic: Epilepsy,Type of session: Case-based Workshop,,N. Gaspard; Brussels/BE,CBW01_02,Coma after cardiac arrest
3,Room: CbW Room,\nDescription\nThis case-based workshop is int...,CbW01 - Continuous EEG in critically ill patients,"Saturday,May23,08:00\n-09:30",Topic: Epilepsy,Type of session: Case-based Workshop,,B. Baykan Baykal; Istanbul/TR,CBW01_03,Miscellaneous aetiologies of coma
4,Room: Main Auditorium,\nDescription\nTreatment of genetic disorders ...,FW01 - Molecular mechanisms for the treatment ...,"Saturday,May23,08:00\n-09:30",Topic: Neurogenetics,Type of session: Focused Workshop,,A. Dürr; Paris/FR,FW01_01,Antisense oligonucleotides
...,...,...,...,...,...,...,...,...,...,...
2220,Room: Amsterdam,\nDescription\nThe speakers will address the s...,TC17 - EAN / WMS: Early diagnosis of and preve...,"Tuesday,May26,13:00\n-16:30",Topic: Muscle and neuromuscular junction disea...,Type of session: Teaching Course,,K. Claeys; Leuven/BE,TC17_04,Antibody-mediated hyperexcitability syndromes
2221,Room: Copenhagen,\nDescription\nThis course aims to improve you...,TC18 - Rare cerebrovascular conditions - Level 2,"Tuesday,May26,13:00\n-16:30",Topic: Cerebrovascular diseases,Type of session: Teaching Course,,M. Krämer; Essen/DE,TC18_01,MOYA-MOYA disease
2222,Room: Copenhagen,\nDescription\nThis course aims to improve you...,TC18 - Rare cerebrovascular conditions - Level 2,"Tuesday,May26,13:00\n-16:30",Topic: Cerebrovascular diseases,Type of session: Teaching Course,,J. Ferro; Lisbon/PT,TC18_02,Cerebral venous thrombosis
2223,Room: Copenhagen,\nDescription\nThis course aims to improve you...,TC18 - Rare cerebrovascular conditions - Level 2,"Tuesday,May26,13:00\n-16:30",Topic: Cerebrovascular diseases,Type of session: Teaching Course,,M. Mancuso; Pisa/IT,TC18_03,MELAS


In [19]:
len(Masterdf)

2225

In [22]:
Masterdf

,Room:,Session Description,Session Heading,Session Time,Topic:,Type of session:,Presentation Abstract,Presentation Authors,Presentation Code,Presentation Title
0,Room: Vienna,\nDescription\nMechanical thrombectomy revolut...,FW02 - Overarching theme - Can we protect and ...,"Saturday,May23,08:00\n-09:30","Topic: Cerebrovascular diseases, Overarching t...",Type of session: Focused Workshop,NaN,NaN,NaN,NaN
1,Room: CbW Room,\nDescription\nThis case-based workshop is int...,CbW01 - Continuous EEG in critically ill patients,"Saturday,May23,08:00\n-09:30",Topic: Epilepsy,Type of session: Case-based Workshop,,S. Beniczky; Dianalund/DK,CBW01_01,Nonconvulsive status epilepticus
2,Room: CbW Room,\nDescription\nThis case-based workshop is int...,CbW01 - Continuous EEG in critically ill patients,"Saturday,May23,08:00\n-09:30",Topic: Epilepsy,Type of session: Case-based Workshop,,N. Gaspard; Brussels/BE,CBW01_02,Coma after cardiac arrest
3,Room: CbW Room,\nDescription\nThis case-based workshop is int...,CbW01 - Continuous EEG in critically ill patients,"Saturday,May23,08:00\n-09:30",Topic: Epilepsy,Type of session: Case-based Workshop,,B. Baykan Baykal; Istanbul/TR,CBW01_03,Miscellaneous aetiologies of coma
4,Room: Main Auditorium,\nDescription\nTreatment of genetic disorders ...,FW01 - Molecular mechanisms for the treatment ...,"Saturday,May23,08:00\n-09:30",Topic: Neurogenetics,Type of session: Focused Workshop,,A. Dürr; Paris/FR,FW01_01,Antisense oligonucleotides
...,...,...,...,...,...,...,...,...,...,...
2220,Room: Amsterdam,\nDescription\nThe speakers will address the s...,TC17 - EAN / WMS: Early diagnosis of and preve...,"Tuesday,May26,13:00\n-16:30",Topic: Muscle and neuromuscular junction disea...,Type of session: Teaching Course,,K. Claeys; Leuven/BE,TC17_04,Antibody-mediated hyperexcitability syndromes
2221,Room: Copenhagen,\nDescription\nThis course aims to improve you...,TC18 - Rare cerebrovascular conditions - Level 2,"Tuesday,May26,13:00\n-16:30",Topic: Cerebrovascular diseases,Type of session: Teaching Course,,M. Krämer; Essen/DE,TC18_01,MOYA-MOYA disease
2222,Room: Copenhagen,\nDescription\nThis course aims to improve you...,TC18 - Rare cerebrovascular conditions - Level 2,"Tuesday,May26,13:00\n-16:30",Topic: Cerebrovascular diseases,Type of session: Teaching Course,,J. Ferro; Lisbon/PT,TC18_02,Cerebral venous thrombosis
2223,Room: Copenhagen,\nDescription\nThis course aims to improve you...,TC18 - Rare cerebrovascular conditions - Level 2,"Tuesday,May26,13:00\n-16:30",Topic: Cerebrovascular diseases,Type of session: Teaching Course,,M. Mancuso; Pisa/IT,TC18_03,MELAS


In [0]:
Masterdf['Session Description'] = Masterdf['Session Description'].str.replace('\n','')

In [0]:
Masterdf['Day'] = Masterdf['Session Time'].str.split(',',expand = True)[0]

In [0]:
Masterdf['Date'] = Masterdf['Session Time'].str.split(',',expand = True)[1]

In [0]:
Masterdf['Start Time'] = Masterdf['Session Time'].str.split(',',expand = True)[2].str.split('\n-',expand = True)[0]
Masterdf['End Time'] = Masterdf['Session Time'].str.split(',',expand = True)[2].str.split('\n-',expand = True)[1]

In [0]:
Masterdf.to_excel('EAN2020.xls')

In [41]:
Masterdf.columns

Index(['Room:', 'Session Description', 'Session Heading', 'Session Time',
       'Topic:', 'Type of session:', 'Presentation Abstract',
       'Presentation Authors', 'Presentation Code', 'Presentation Title',
       'Day', 'Date', 'Start Time', 'End Time'],
      dtype='object')

In [0]:
Columns = ['Session Heading','Room:', 'Session Description', 'Session Time','Day', 'Date', 'Start Time', 'End Time',
       'Topic:', 'Type of session:', 'Presentation Code', 'Presentation Title', 'Presentation Abstract',
       'Presentation Authors'
       ]

In [44]:
len(Columns)

14

In [45]:
len(Masterdf.columns)

14

In [0]:
Masterdf = Masterdf[Columns]